In the following notebook a rough idea for the calculation of a sensitivity curve for a potential acoustic neutrino telescope will be drawn up. This will be done following a paper by the KM3Net collaboration.

https://www.km3net.org/wp-content/uploads/2023/05/KM3NeT_DS_TDR-published-in-2010.pdf

In [3]:
from scipy.integrate import quad
from scipy.integrate import simps
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import numpy as np

-------------------------------------------------------------------------------------------------------------------------------
In this notebook the general code for calculating the sensitivity of the detector will be tried out with different initial assumption that simulate a very good functionality.

# Assumptions

* The detector is able to reconstruct 99 out of 100 neutrino events
* The neutrino comes from straight above without any inclination



-------------------------------------------------------------------------------------------------------------------------------



The first thing needed for a sensitivity curve is the *Detector Effective Volume* ($V_{eff}(E_{\nu}, \theta_{\nu})$). This is defined by the following formula:

$$ V_{eff}(E_{\nu}, \theta_{\nu}) = \frac{N_{rec}(E_{\nu}, \theta_{\nu})}{N_{gen}(E_{\nu}, \theta_{\nu})} * V_{gen}$$

**Parameters:**
* $N_{rec}(E_{\nu}, \theta_{\nu})$ = number of reconstructed events
* $N_{gen}(E_{\nu}, \theta_{\nu})$ = neutrino interactions in $V_{gen}$
* $V_{gen}$ = generation volume

In [2]:
def detector_effective_volume(Nrec, Ngen, Vgen):
    
    # plug in the above formula
    Veff = (Nrec / Ngen) * Vgen
        
    return Veff

In [1]:
# Say that you find 99 of 100 neutrino events and can track them accurately
Nrec = 99
Ngen = 100

# Make the generation volume 100 km^3 or 100000000000 m^3
Vgen = 100000000000

Then the *Neurtino Effective Area* ($A_{eff}^{\nu}(E_{\nu}, \theta_{\nu})$) can be calculated with the following formula: 

$$A_{eff}^{\nu}(E_{\nu}, \theta_{\nu}) = V_{eff}(E_{\nu}, \theta_{\nu}) \times \rho N_A \times \sigma(E_{\nu}) \times P_{earth}(E_{\nu}, \theta_{\nu})$$

**Parameters:**
* $A_{eff}^{\nu}(E_{\nu}, \theta_{\nu})$ = neutrino effective area
* $V_{eff}(E_{\nu}, \theta_{\nu})$ = detector effective formula calculated above
* $\rho N_A$ = number of nucleons per unit volume ($\rho$ = effective matter density [$\frac{mol}{V}$], $N_A$ = Avogadro number)
* $\sigma(E_{\nu})$ = neutrino cross section in the reaction channel considered
* $P_{earth}(E_{\nu}, \theta_{\nu})$ = neutrino transmission probability through the Earth

In [4]:
def neutrino_effective_area(Nrec, Ngen, Vgen, rho, Na, sigma, Pearth):
    
    # define the detector effective volume
    Veff = detector_effective_volume(Nrec, Ngen, Vgen)
    
    # plug in the above formula
    Aeff = Veff * rho * Na * sigma * Pearth
        
    return Aeff

The *Neutrino Transmission Probability* is the likelihood that a neutrino of a certain energy and initial flavor, traveling through the Earth's matter, will retain its flavor or change to a different flavor before being detected. In the SeaGen code this quantity is formulised as the following:

$$P_{earth}(E_{\nu}, cos θ_{\nu}) = e^{−N_A*\sigma(E_{\nu})* \rho_l(θ_{\nu})}$$

**Parameters:**
* $P_{earth}(E_{\nu}, cos θ_{\nu})$ = the neutrino transmission probability through earth
* $N_A$ = Avogadro's number
* $\sigma(E_{\nu})$ = neutrino cross section in the reaction channel considered
* $\rho_l(θ_{\nu})$ = is the amount of material encountered by a neutrino in its passage through the Earth

https://arxiv.org/pdf/1602.00501.pdf

In [5]:
def neutrino_transmission_probability(Na, sigma):
    
    # find the material to cross in earth
    # material = amount_of_material_earth(rho_earth, L)
    
    # perform the function
    Pearth = np.exp**(-Na * sigma * material)
    
    return Pearth

The amount of material encountered in earth can be determined with the following formula:

$$\rho_l(θ_{\nu}) = \int_{L} \rho_{earth(r)} dl$$

**Parameters:**
* $L$ = neutrino path at angle $\theta_{\nu}$
* $\rho_{earth(r)}$ = PREM Model of the earth following https://lweb.cfa.harvard.edu/~lzeng/papers/PREM.pdf

In [14]:
def amount_of_material_earth(rho_earth, L):

    # perform the integral
    result, _ = spi.quad(rho_earth, 0, L)

    return result

In [ ]:
# the neutrino's travel through air first and then through water [kg/m^3] and [km]
density_air_bottom = 1.225 
density_air_top = 0
L_atmosphere = 10000 

density_water_bottom = 998
density_water_top = 997
L_water = 1.5

In [7]:
# so we can say that the amount_of_material_earth is equal to the following
material = 10000000 * 0.612 + 1500 * 997.5

# the units of material are kg/m^2
print(material)

7616250.0


In [8]:
# define Avogadro's constant [mol^-1]
Na = 6.02214076 * 10**(23)

In [15]:
# find Veff with the above defined formula
Veff = detector_effective_volume(Nrec, Ngen, Vgen)

# define the density of water at a depth of 1500 m [g/cm^3]
density = 0.998
# get the molar mass of water [g/mol]
molar_mass = 18.01528 
# the matter density is then [mol/cm^3]
rho = density / molar_mass

# http://cupp.oulu.fi/neutrino/nd-cross.html
sigma = ???

SyntaxError: invalid syntax (4071919507.py, line 12)

Lastly, to find the *Number of Detectable Events* ($N_s$) we can use the following formula:

$$N_s = \int \int A_{eff}^{\nu}(E_{\nu}, \theta_{\nu}) \frac{d\phi_{\nu}}{dE_{\nu}d\theta_{\nu}} dE_{\nu}d\theta_{\nu}$$

**Parameters:**
* $N_s$ = number of detectable events
* $A_{eff}^{\nu}(E_{\nu}, \theta_{\nu})$ = neutrino effective area calculated above
* $\frac{d\phi_{\nu}}{dE_{\nu}d\theta_{\nu}}$ = predicted astrophysical flux 

First we need a function for the predicted astrophysical flux. This we can find in the following source where they approximatethe flux to be:

$$(\frac{d\phi_{\nu}}{dE_{\nu}})_{limit} = \frac{1 - 4 \times 10^{-8}}{E_{\nu}^2} GeV cm^{-2} s^{-1} sr^{-1}$$

We also know:

$$ \frac{d\phi_{\nu}}{dE_{\nu}d\theta_{\nu}} = \frac{d\phi_{\nu}}{dE_{\nu}} * \frac{1}{\Omega} $$

https://iopscience.iop.org/article/10.1086/340281/pdf

In [7]:
def predicted_astrophysical_flux(E, solid_angle):
    # (dphi)/(dE dtheta)
    dphi_dEdtheta = (4 * 10**(-8) / E**2) * (1 / solid_angle) 
    return dphi_dEdtheta

In [ ]:
# calculate the solid angle of the detector
solid_angle = ???

In [8]:
def number_of_detectable_events(Nrec, Ngen, Vgen, rho, Na, sigma, Pearth, E, solid_angle):
    
    # define the function to be integrated
    def integrand(Nrec, Ngen, Vgen, rho, Na, sigma, Pearth, E, solid_angle):
        
        # define the neutrino effective area
        Aeff = neutrino_effective_area(Nrec, Ngen, Vgen, rho, Na, sigma, Pearth)
        
        # define the neutrino effective area
        astrophysical_flux = predicted_astrophysical_flux(E, solid_angle)
        
        return Aeff * astrophysical_flux

    # Define the limits of integration for E_nu and theta_nu
    E_nu_min = 10**(16)  
    E_nu_max = 10**(21)
    theta_nu_min = ???
    theta_nu_max = ???

    # Calculate the integral
    result, _ = dblquad(integrand, theta_nu_min, theta_nu_max, lambda _: E_nu_min, lambda _: E_nu_max)

    return result

SyntaxError: invalid syntax (1283706790.py, line 17)

## Things to find out

#### Parameters
* Nrec 
* Ngen 
* Vgen 
* L
* rho_earth
* sigma
* solid_angle